# Närvivõrk ussi mängimiseks
Üldine idee lähenemiseks võetud: https://towardsdatascience.com/today-im-going-to-talk-about-a-small-practical-example-of-using-neural-networks-training-one-to-6b2cbd6efdb3

Ühel ajahetkel on ussil võimalik minna kas üles, alla, vasakule või paremale kuid kõik nendest valikutest ei ole muidugi mõistlikud. Otsustamise hetkel annab närvivõrk igale käigule hinnangu vastavalt:  
- -1 kui selle käigu järel uss sureks;   
- 0 kui käigu järel uss ei sure, aga ei liigu ka võidule lähemale;  
- 1 kui uss jääb ellu ja on käigu tulemusel liikunud toidule lähemale;



# Impordid

In [1]:
# Numpy
import numpy as np

# Keras
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
# Pandas
import pandas as pd

# Sklearn
import sklearn.preprocessing
import sklearn.utils

# Other
import random
import itertools
from functools import reduce
import math

# Board dimensions
BOARD_WIDTH = 20
BOARD_HEIGHT = 20

# Õppimisandmete genereerimine

In [3]:
UP, DOWN, LEFT, RIGHT = ((0,-1), (0,1), (-1,0), (1,0))


class Snake:
    def __init__(self, id, position):
        self.id = id
        
        self.lastDirection = lambda :(position[-1][0]-position[-2][0], position[-1][1]-position[-2][1])
        # List where, position[0] == tail, position[-1] == head
        self.position = position
        
        for pos in position:
            BOARD[pos[1]][pos[0]] = id
        
        # Function which gets called when the snake needs to make a move.
        self.moveDecider = self.randomMove
        #self.moveDecider = self.ANN_Move
        self.head = lambda : position[-1]
        
        
    # All possible moves for snake, that, dont hit itself.
    def possibleMoves(self, startPos):
        moves = []
        # Array of all occupied positions
        occupied = reduce(lambda a,b: a+b, [snake.position for snake in SNAKES])

        for move in (UP, DOWN, LEFT, RIGHT):
            frm = startPos
            new_x = (frm[0] + move[0]) % BOARD_WIDTH
            new_y = (frm[1] + move[1]) % BOARD_HEIGHT

            # Väldib endasse ja teise snek-i sisse sõitmist
            if (new_x, new_y) not in occupied:
                moves.append(move)

        return moves
    
    # Returns a random move from possibleMoves()
    def randomPossibleMove(self):
        moves = self.possibleMoves(self.position[-1])
        # If no move is possible, choose random
        if len(moves) == 0:
            moves = [UP, DOWN, LEFT, RIGHT]
        return random.choice(moves)
    
    def randomMove(self):
        return random.choice([UP, DOWN, LEFT, RIGHT])
    
    def ANN_Move(self):
        coordinatesAroundHead = [((self.head()[0] + direction[0])%BOARD_WIDTH, (self.head()[1] + direction[1])%BOARD_WIDTH) for direction in [UP, DOWN, LEFT, RIGHT]]
        # 0/1 whether position is taken/open [UP, DOWN, LEFT, RIGHT]
        openCoordinatesAroundHead = [1 if BOARD[pos[1]][pos[0]] in (None, 0) else 0 for pos in coordinatesAroundHead]
        directionToApple = math.atan2(FOOD_LOC[1]-self.head()[1], FOOD_LOC[0]-self.head()[0]) / math.pi
        distanceFromApple = (FOOD_LOC[0]-self.head()[0]) + (FOOD_LOC[1]-self.head()[1])
        
        moves = [UP, DOWN, LEFT, RIGHT]
        movementChoices = []
        for move in moves:
            movementChoices.append([*openCoordinatesAroundHead, directionToApple, *move])
        
        predictions = model.predict(movementChoices)
        bestMove = moves[np.argmax(predictions)]
        
        return bestMove
    
    # Moves the snake based on its moveDecider
    def move(self):
        direction = self.moveDecider()
        self.lastDirection = direction
        head = self.position[-1]

        # Direction coordinates get added to head.
        # % - if coordinate exceeds board size, start from other end
        new_x = (head[0] + direction[0]) % BOARD_WIDTH
        new_y = (head[1] + direction[1]) % BOARD_HEIGHT
        new_head = (new_x, new_y)
        self.position.append(new_head)
        
        # If snake hits food, dont remove tail block
        if BOARD[new_y][new_x] == 0:
            tail = self.position[0]
            generateFood()
        else:
            tail = self.position.pop(0)

        # BOARD changes 
        # Adds new head to board
        BOARD[new_head[1]][new_head[0]] = self.id
        # Removes tail from board
        BOARD[tail[1]][tail[0]] = None
        return direction, gameOver()

def generateFood():
    global FOOD_LOC
    options = []
    for x in range(BOARD_WIDTH):
        for y in range(BOARD_HEIGHT):
            if BOARD[y][x] == None:
                options.append((x,y))
    food_loc = random.choice(options)

    BOARD[food_loc[1]][food_loc[0]] = 0

    FOOD_LOC = food_loc

def generateSnake():
    head = (random.randint(0, 19),random.randint(0, 19))
    direction = (random.choice([UP, DOWN, LEFT, RIGHT]))
    body = [head]
    for i in range(4):
        body.append(((body[-1][0]+direction[0])%BOARD_WIDTH, (body[-1][1]+direction[1])%BOARD_HEIGHT))
    
    return Snake(1, body[::-1])
    
# Cheks if any snakes have overlapping bodyparts
def gameOver():
    # Board is full
    if not any(None in row for row in BOARD):
        return True
    bodies = reduce(lambda a,b: a+b, [snake.position for snake in SNAKES])
    return len(bodies) != len(set(bodies)) 



scores = []
# Stores moves
gamedata = []
for counter in range(1000000):
#counter = 0
#while True:
    # Game instance setup
    BOARD = [[None for x in range(BOARD_WIDTH)] for y in range(BOARD_HEIGHT)]
    FOOD_LOC = None
    SNAKES = []
    #snake = Snake(1, [(1,1), (2,1), (3,1), (4,1), (5,1)])
    snake = generateSnake()
    SNAKES.append(snake)
    generateFood()
        
    gameRound = 0
    gameOverFlag = False
    # Checks whether snake has run into itself, board is full or round has exceded a reasonable amount of moves
    while not gameOverFlag and gameRound < 5000:
        ############ 
        #Data for training ANN
        
        head = snake.position[-1]
        coordinatesAroundHead = [((head[0] + direction[0])%BOARD_WIDTH, (head[1] + direction[1])%BOARD_WIDTH) for direction in [UP, DOWN, LEFT, RIGHT]]
        # 0/1 whether position is taken/open [UP, DOWN, LEFT, RIGHT]
        openCoordinatesAroundHead = [1 if BOARD[pos[1]][pos[0]] in (None, 0) else 0 for pos in coordinatesAroundHead]
        #directionToApple = math.atan2(FOOD_LOC[1]-head[1], FOOD_LOC[0]-head[0]) / math.pi
        distanceFromApple = (FOOD_LOC[0]-head[0]) + (FOOD_LOC[1]-head[1])
        
        vectorToApple = [None, None]
        d_x = FOOD_LOC[0]-snake.head()[0]
        d_y = FOOD_LOC[1]-snake.head()[1]
        if d_x > 0:
            vectorToApple[0] = 1
        elif d_x < 0:
            vectorToApple[0] = -1
        else:
            vectorToApple[0] = 0
            
        if d_y > 0:
            vectorToApple[1] = 1
        elif d_y < 0:
            vectorToApple[1] = -1
        else:
            vectorToApple[1] = 0
        
        ############
        # Moving snake
        
        OLD_FOOD_LOC = FOOD_LOC
        move, gameOverFlag = snake.move()
        
        ############ 
        # Data for training ANN
        
        # -1 if snake died
        # 0 if snake survived
        # 1 if snake survived and moved closer to food
        resultMetric = None
        if gameOverFlag:
            resultMetric = -1
        else:
            newDistanceFromApple = (OLD_FOOD_LOC[0]-snake.head()[0]) + (OLD_FOOD_LOC[1]-snake.head()[1])
            if newDistanceFromApple >= distanceFromApple:
                resultMetric = 0
            else:
                resultMetric = 1
                
        #gamedata.append([[*openCoordinatesAroundHead, directionToApple, *move], resultMetric])
        gamedata.append([[*openCoordinatesAroundHead, *vectorToApple, *move], resultMetric])
        #gamedata.append([[*vectorToApple, *move], resultMetric])
        
        
        
        ############
        gameRound += 1
    
    scores.append(len(snake.position))
    
    if counter % 100000 == 0:
        print("Done: {:,}".format(counter), end="\r")
    counter += 1
    
print("Highest score:", max(scores))
print("Movecount:", len(gamedata))

Highest score: 7
Movecount: 3651175


# Mudel

Katsetasin suure hulga mudeleid erinevate kihtide ja parameetritega, aga ükski ei viinud sihini.  
Siia jääb viimane toimiv mudel.

In [4]:
X = np.asarray([np.asarray(x[0]) for x in gamedata])
y = np.asarray([x[1] for x in gamedata])

import tensorflow as tf
# Quick hack to fix GPU memory allocation
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#tf.keras.backend.clear_session()
        
model = Sequential() 

# , activation="relu"
model.add(Dense(16, input_shape=(len(X[0]),)))
model.add(Dense(32))
model.add(Dense(1))

model.compile(loss='mse', metrics=['mae'], optimizer=optimizers.RMSprop(lr=0.01))
model.summary()

with tf.device('/gpu:0'):
    model.fit(X, y, epochs=5, batch_size=1024, validation_split = 0.1, )

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                144       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 721
Trainable params: 721
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
3210/3210 [==============================] - 6s 2ms/step - loss: 0.5010 - mae: 0.6007 - val_loss: 0.5013 - val_mae: 0.5713
Epoch 2/5
3210/3210 [==============================] - 6s 2ms/step - loss: 0.4975 - mae: 0.5982 - val_loss: 0.5027 - val_mae: 0.6293
Epoch 3/5
3210/3210 [==============================] - 6s 2ms/step - loss: 0.4973 - mae: 0.5979 - val_loss: 0.5273 - val_m

## Mudeli salvestamine

In [6]:
from tensorflow.keras.models import load_model
model.save("model.h5")

uusMudel = load_model("model.h5")
print(uusMudel.summary())
#uusMudel.fit(X, y, epochs=10, batch_size=8192, validation_split = 0.1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                144       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 721
Trainable params: 721
Non-trainable params: 0
_________________________________________________________________
None


## Varia (jooksvalt ennustuste uurimine jms.)

In [7]:
predictions = uusMudel.predict(X)
count = -1
deviating = [a for a,b in zip(y, predictions) if abs(a-b)>0.5]
print(len(y))
print(len(deviating))

3651175
1798239
